# Обучаем первые классификаторы в sklearn

### Данные


По данным характеристикам молекулы требуется определить, будет ли дан биологический ответ (biological response).

Для демонстрации используется обучающая выборка из исходных данных bioresponse.csv, файл с данными прилагается.

### Готовим обучающую и тестовую выборки

In [1]:
import pandas as pd

bioresponce = pd.read_csv('bioresponse.csv', header=0, sep=',')

In [2]:
bioresponce.head(5)

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [3]:
y = bioresponce.Activity.values

In [4]:
X = bioresponce.iloc[:, 1:]

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Строим модель и оцениваем качество

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
model = LogisticRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)

In [8]:
type(preds)

numpy.ndarray

In [9]:
10 // 9

1

In [10]:
print(sum(preds == y_test) / len(preds))

0.7560581583198708


In [11]:
print(sum(preds == y_test) / float(len(preds)))

0.7560581583198708


In [12]:
from sklearn.metrics import accuracy_score

print(accuracy_score(preds, y_test))

0.7560581583198708


### Качество на кросс-валидации

In [13]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(model, X_train, y_train, cv=5))

[0.74404762 0.73956262 0.72310757 0.75099602 0.75896414]


In [14]:
print(cross_val_score(model, X_train, y_train, cv=5).mean())

0.7433355944771515


### Пробуем другие классификаторы

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


In [16]:
%%time

models = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    LinearSVC(),
    RandomForestClassifier(n_estimators=100), 
    GradientBoostingClassifier(n_estimators=100)
]

for model in models:
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(accuracy_score(preds, y_test), model)

0.7189014539579968 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
0.7075928917609047 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
0.7407108239095315 LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
0.7802907915993538 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split

## Опциональное задание:

Попробуйте разные классификаторы с разными параметрами и постарайтесь добиться максимального качества на тестовой выборке

In [18]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [19]:
models = {'LogisticRegression': LogisticRegression(),
          'KNeighborsClassifier': KNeighborsClassifier(),
          'DecisionTreeClassifier': DecisionTreeClassifier(),
          'LinearSVC': LinearSVC(),
          'RandomForestClassifier': RandomForestClassifier(), 
          'GradientBoostingClassifier': GradientBoostingClassifier()}

In [20]:
tuned_parameters = {'LogisticRegression' :  {'C': [0.1, 1, 10], \
                                             'max_iter': [10, 20], \
                                             'solver': ['newton-cg', 'lbfgs'], \
                                             'warm_start': [True, False]},
                    'KNeighborsClassifier' : {'n_neighbors': np.arange(1, 5, 2), \
                                              'metric': ['euclidean', 'cityblock'], \
                                              'leaf_size': [10, 30], \
                                              'algorithm': ['auto', 'ball_tree']},
                    'DecisionTreeClassifier' : {'max_depth': [3, None], \
                                                'max_features': np.arange(1, 4), \
                                                'min_samples_leaf': np.arange(1, 4), \
                                                'criterion': ['gini', 'entropy']},
                    'LinearSVC' : {'C': [0.001, 0.01, 0.1, 1, 10], \
                                   'intercept_scaling': [0.001, 0.01, 0.1, 1], \
                                   'intercept_scaling': [0.1, 1], \
                                   'max_iter': [100, 1000]},
                    'RandomForestClassifier' : {'n_estimators': [9, 18, 27, 36, 45, 54, 63], \
                                                'max_depth': [1, 5, 10, 15, 20, 25, 30], \
                                                'min_samples_leaf': [1, 2, 4, 6, 8, 10], \
                                                'warm_start': [True, False]},
                    'GradientBoostingClassifier' : {'learning_rate': [0.1, 0.5], \
                                                    'max_depth': [4, 6], \
                                                    'min_samples_leaf': [5, 10], \
                                                    'max_features': [1.0, 0.5, 0.1]}}

In [21]:
for key, model in models.items():
    clf = GridSearchCV(estimator=model, param_grid=tuned_parameters[key], n_jobs=-1)
    clf.fit(X_train, y_train)
    print(clf.best_score_)
    print(clf.best_estimator_)

D:\Programs\Anaconda2\envs\py35\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


0.7481098288897732
LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=True)
0.7302029446876244
KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='cityblock',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')
0.6653402307998408
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=3, max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
0.7926780740151214
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min

In [22]:
for key, model in models.items():
    clf = RandomizedSearchCV(model, param_distributions=tuned_parameters[key], n_jobs=-1)
    clf.fit(X_train, y_train)
    print(clf.best_score_)
    print(clf.best_estimator_)

0.7465181058495822
LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=20, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=True)
0.7302029446876244
KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='cityblock',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')
0.6633505769996021
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=1, max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
0.7767608436132113
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_sam

##### __GridSearchCV__ и __RandomizedSearchCV__ анализируют один и тот же набор гиперпараметров. Результат работы  __RandomizedSearchCV__ несколько хуже, однако время работы __GridSearchCV__ значительно больше. В случае большого количества различных параметров выгоднее использовать  __RandomizedSearchCV__, для более детального анализа лучше подходит __GridSearchCV__.